In [39]:
import pandas as pd
import datetime
import numpy as np
import os
import os.path

now = datetime.datetime.now()
today = datetime.date.today()
strtoday = now.strftime('%d-%m-%Y')
yesterday = now-datetime.timedelta(days=1)
yday = yesterday.strftime('%d-%m-%Y')

df = pd.read_excel("D2C_database.xlsx",sheet_name='source_data')
df['date']=pd.to_datetime(df['date'],format='%d-%m-%Y')

#df['date']=pd.to_datetime(df['date'],format='%Y-%m-%d %H:%M:%S' ).dt.strftime('%d-%m-%Y')
df['mobile_number']=df['mobile_number'].astype(str)
df=df.sort_values(by='date',ascending=False)

for no in df['mobile_number'].unique():
    first_name=df[df['mobile_number']== no].iloc[0,1]
    df.loc[df['mobile_number']==no,'customer_name']=first_name
    address = df[df['mobile_number'] == no].iloc[0, 2]
    df.loc[df['mobile_number'] == no, 'address'] = address

df_ref = pd.DataFrame(columns = ["customer_name","address","mobile_number","Min_Date","Max_Date","min_diff","max_diff","no of orders",'yesterday','count','order_freq_days(avg)'])
df_ref['customer_name']=df['customer_name']
df_ref['address']=df['address']
df_ref['mobile_number']=df['mobile_number']
df_ref['yesterday']=yesterday

for no in df['mobile_number'].unique():
    max_date=df[df['mobile_number']== no].iloc[0,0]
    min_date=df[df['mobile_number']== no].iloc[-1,0]
    diff_date=set(df[df['mobile_number']== no]['date'].drop_duplicates().diff().abs().dt.days)
    diff_date = [x for x in diff_date if str(x) != 'nan']
    df_ref.loc[df_ref['mobile_number']==no,'min_diff']=min(diff_date)
    df_ref.loc[df_ref['mobile_number']==no,'max_diff']=max(diff_date)
    n_orders=len(df[df['mobile_number']== no]['date'].unique())
    df_ref.loc[df_ref['mobile_number']==no,'Min_Date']=min_date
    df_ref.loc[df_ref['mobile_number']==no,'Max_Date']=max_date
    df_ref.loc[df_ref['mobile_number']==no,'no of orders']=n_orders
    df_ref.loc[df_ref['mobile_number']==no,'order_freq(avg)']=(max(diff_date)-min(diff_date))/n_orders
    
#max_date=max_date.to_pydatetime()
#df_ref['Max_Date']=df_ref['Max_Date'].datetime.now().date()

df_ref['yesterday']=pd.to_datetime(df_ref['yesterday'])
#print(df_ref['Max_Date'])
#print(df_ref['yesterday'])
#print(type(df_ref['yesterday']))
#print(type(df_ref['Max_Date']))
df_ref['Max_Date']=pd.to_datetime(df_ref['Max_Date'],format='%Y-%m-%d %H:%M:%S' )
df_ref['diff'] = (df_ref['yesterday']-df_ref['Max_Date']).dt.days
df_ref.loc[df_ref['diff']>=30,'count']='Inactive'
df_ref.loc[df_ref['diff']<30,'count']='Active'

#f_customer_db = pd.DataFrame()

#inactive_Users = df_ref.loc[df_ref['count'] == 'Inactive','count'].count()

#active_Users = df_ref.loc[df_ref['count'] == 'Active','count'].count()

#print(df_ref[df_ref['diff'] < 30].index)
df_customers=pd.read_excel("customer_list.xlsx",sheet_name='customer_list' )

df_ref=df_ref.drop_duplicates(subset=['mobile_number'])
active_Users = len(df_ref[df_ref['diff'] < 30].index)
inactive_Users = len(df_ref[df_ref['diff'] >= 30].index)
dormant_customers = df_customers.loc[df_customers['Total order value'] == 'None','Total order value'].count()
customer_db_series = pd.Series([inactive_Users,active_Users,dormant_customers], index =['Inactive_Users','Active_Users','Dormant_Users'])
df=df.sort_values(by='date',ascending=True)
df['date']=df['date'].dt.strftime('%d-%m-%Y')
writer = pd.ExcelWriter('D2C_database_op.xlsx',
        engine='xlsxwriter' )
workbook = writer.book

df.to_excel( writer, sheet_name='database',index=False )
df_ref.to_excel( writer, sheet_name='Customer_status' ,index=False)
customer_db_series.to_excel( writer, sheet_name='Customer_list_Db')
writer.save()

In [5]:
df_ref

,customer_name,address,mobile_number,Min_Date,Max_Date,no of orders,yesterday,count,diff
65154,"#45, Jaya Splendour, flat no. 303, 1st main, ...",160,919945360140,50806689,1970-01-01 00:00:00.050806689,1,2020-09-08 14:31:45.103859,Inactive,18513
64676,"147, 1st cross 1st main, Isro layout",16,919900297214,50741486,1970-01-01 00:00:00.050741486,1,2020-09-08 14:31:45.103859,Inactive,18513
64674,"#77, ANKURA, 1st Main, 2nd Stage, Arakere Mic...",30,919448076580,50738716,1970-01-01 00:00:00.050738716,1,2020-09-08 14:31:45.103859,Inactive,18513
64654,"Sumadhura Silver Ripples B701, Borewel road ,...",45,919739762446,3608,1970-01-01 00:00:00.050738209,6,2020-09-08 14:31:45.103859,Inactive,18513
64790,"G-13, Durga Saffron Square, OPP INTEL, DEVERA...",35,919886596610,50761890,1970-01-01 00:00:00.050761890,1,2020-09-08 14:31:45.103859,Inactive,18513
64630,"SGR GREEN VIEW APARTMENT, 103, 4A/2, RAMIAH R...",22.5,919886452657,50736352,1970-01-01 00:00:00.050736352,1,2020-09-08 14:31:45.103859,Inactive,18513
64624,"Apartment 7, 2nd Floor, 545, Matru Chaya, 22n...",15,919880879769,50735663,1970-01-01 00:00:00.050735663,1,2020-09-08 14:31:45.103859,Inactive,18513
64767,"A303 Sriven Splendour, Rama Temple Road Chall...",20,919886741895,3745,1970-01-01 00:00:00.050755805,2,2020-09-08 14:31:45.103859,Inactive,18513
64756,"No.28, 6th Cross, UAS Layout,, Sanjaynagar",230,919845142471,50747949,1970-01-01 00:00:00.050747949,1,2020-09-08 14:31:45.103859,Inactive,18513
64720,"B7 904,L&T south city, Arekere Mico Layout",16,919741387666,3387,1970-01-01 00:00:00.050745788,7,2020-09-08 14:31:45.103859,Inactive,18513


In [31]:
df[df['mobile_number']== '919831059991']

,Order ID,address,amount,customer_name,date,mobile_number,qty,sku
6453,542,22.5,6.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,0.3,Red Amarantus.(GT)
6449,542,22.5,4.50,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,0.3,Spinach(GT)
6451,542,22.5,160.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,4.0,Potato
6448,542,22.5,6.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,0.3,Methi(GT)
6452,542,22.5,88.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,1.6,Papaya
6450,542,22.5,6.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,0.3,Green Amarantus(GT)
6447,542,22.5,20.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,0.3,Coriander (GT)
6454,542,22.5,4.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,0.2,Spring Onion (GT)
6446,542,22.5,180.00,"209 Kapila Towers, RBI Colony Koramangala",19-04-2020,919831059991,4.0,Watermelon Kiran
11430,948,22.5,60.00,"209 Kapila Towers, RBI Colony Koramangala",27-04-2020,919831059991,1.0,Ridgegourd


In [49]:
diff_date = [x for x in diff_date if str(x) != 'nan']
max(diff_date)

33.0